# Senate Voting Test

_James Weichert_

In [257]:
import pandas as pd
import numpy as np
import bpr

import chart_studio.plotly as pl
import plotly.graph_objects as go
from ipywidgets import widgets

In [258]:
current_senate = pd.read_csv('party_data.csv', index_col=0)
new_senate = pd.read_csv('new_senate.csv', index_col=0)

In [259]:
current_senate.head()

,State,Population (2018),U.S. House Seats,Senators,PVI,D_share,R_share
0,AL,4887871,7,2,-14,43.0,57.0
1,AK,737438,1,2,-9,45.5,54.5
2,AZ,7171646,9,2,-5,47.5,52.5
3,AR,3013825,4,2,-15,42.5,57.5
4,CA,39557045,53,2,12,56.0,44.0


In [260]:
new_senate.head()

,State,Population (2018),Senators,PVI,D_share,R_share
0,AL,4887871,1,-14,43.0,57.0
1,AK,737438,1,-9,45.5,54.5
2,AZ,7171646,2,-5,47.5,52.5
3,AR,3013825,1,-15,42.5,57.5
4,CA,39557045,12,12,56.0,44.0


In [261]:
def simulate_election(data, turnout=0.56, turnout_variability = 0.05, voting_age_pop=0.77, swing_vote_percent=0.01):
    
    data['D_sen'] = 0
    data['R_sen'] = 0

    for state in range(50): 
        
        registered_voters = int(data.loc[state, 'Population (2018)'] * voting_age_pop)
        
        for seat in range(data.loc[state, 'Senators']):
            
            election_turnout = turnout + np.random.choice(np.arange(-1 * turnout_variability, turnout_variability, turnout_variability/10))
            voters = registered_voters * election_turnout
            
            swing_vote_preference = np.random.choice(np.arange(-1, 1, 0.1))
            
            d_vote_share = int((voters * (1 - swing_vote_percent)) * (data.loc[state, 'D_share'] / 100) + swing_vote_preference * swing_vote_percent * voters)
            r_vote_share = int(voters - d_vote_share)
            
            if d_vote_share > r_vote_share:
                data.loc[state, 'D_sen'] += 1
            elif r_vote_share > d_vote_share:
                data.loc[state, 'R_sen'] += 1
            else:
                r = np.random.rand()
                
                if r < 0.5:
                    data.loc[state, 'D_sen'] += 1
                else:
                    data.loc[state, 'R_sen'] += 1
    
    data['Senate_Scale'] = data['R_sen'] / data['Senators']
    
    return data

In [262]:
curr_senate_voted = simulate_election(current_senate)
curr_senate_voted

,State,Population (2018),U.S. House Seats,Senators,PVI,D_share,R_share,D_sen,R_sen,Senate_Scale
0,AL,4887871,7,2,-14,43.0,57.0,0,2,1.0
1,AK,737438,1,2,-9,45.5,54.5,0,2,1.0
2,AZ,7171646,9,2,-5,47.5,52.5,0,2,1.0
3,AR,3013825,4,2,-15,42.5,57.5,0,2,1.0
4,CA,39557045,53,2,12,56.0,44.0,2,0,0.0
5,CO,5695564,7,2,1,50.5,49.5,2,0,0.0
6,CT,3572665,5,2,6,53.0,47.0,2,0,0.0
7,DE,967171,1,2,6,53.0,47.0,2,0,0.0
8,FL,21299325,27,2,-2,49.0,51.0,0,2,1.0
9,GA,10519475,14,2,-5,47.5,52.5,0,2,1.0


In [263]:
def check_composition(data):
    
    d_senate_num = np.sum(data['D_sen'])
    r_senate_num = np.sum(data['R_sen'])
    senate_total = np.sum(data['Senators'])
    
    print("Democratic Senators:", d_senate_num)
    print("Republican Senators:", r_senate_num)
    print("Senate Total:", senate_total)
    return

In [264]:
check_composition(curr_senate_voted)

Democratic Senators: 38
Republican Senators: 62
Senate Total: 100


In [265]:
vap = widgets.FloatSlider(
    value = 0.77,
    min=0.01,
    max=1,
    step=0.01,
    description='Voting Pop:',
    continuous_update=False
)

In [266]:
turnout = widgets.FloatSlider(
    value = 0.56,
    min=0.0,
    max=1.0,
    step=0.01,
    description='Turnout:',
    continuous_update=False
)

In [267]:
turnout_var = widgets.FloatSlider(
    value = 0.05,
    min=0.0,
    max=1.0 - turnout.value,
    step=0.01,
    description='Turnout Var:',
    continuous_update=False
)

In [268]:
swing_vote_percentage = widgets.FloatSlider(
    value = 0.05,
    min=0.0,
    max=0.50,
    step=0.01,
    description='Swing %:',
    continuous_update=False
)

In [269]:
g = go.FigureWidget(
    data = go.Choropleth(
        locations = current_senate['State'],
        z = current_senate['Senate_Scale'].astype(float),
        locationmode = 'USA-states',
        colorscale = bpr.safe_tossup,
        colorbar_title = 'Test'
    )
)

g.update_layout(
    title_text = 'U.S. Senate',
    geo_scope = 'usa',
    dragmode = False
)

FigureWidget({
    'data': [{'colorbar': {'title': {'text': 'Test'}},
              'colorscale': [[0.0, '#444…

In [270]:
def response(change):
    elected_sen = simulate_election(data, turnout=turnout.value, turnout_variability=turnout_var.value, voting_age_pop=vap.value, swing_vote_percent=swing_vote_percentage.value)
    with g.batch_update():
            g.data[0].z = elected_sen['Senate_Scale']

In [271]:
vap.observe(response, names="value")
turnout.observe(response, names="value")
turnout_var.observe(response, names="value")
swing_vote_percentage.observe(response, names="value")

In [272]:
container = widgets.VBox([vap, turnout, turnout_var, swing_vote_percentage])

In [273]:
widgets.VBox([container])

In [283]:
?go.layout.Slider()

In [279]:
g.update_layout(
    sliders=[vap]
)

ValueError: 
    Invalid element(s) received for the 'sliders' property of layout
        Invalid elements include: [FloatSlider(value=1.0, continuous_update=False, description='Voting Pop:', max=1.0, min=0.01, step=0.01)]

    The 'sliders' property is a tuple of instances of
    Slider that may be specified as:
      - A list or tuple of instances of plotly.graph_objs.layout.Slider
      - A list or tuple of dicts of string/value properties that
        will be passed to the Slider constructor

        Supported dict properties:
            
            active
                Determines which button (by index starting from
                0) is considered active.
            activebgcolor
                Sets the background color of the slider grip
                while dragging.
            bgcolor
                Sets the background color of the slider.
            bordercolor
                Sets the color of the border enclosing the
                slider.
            borderwidth

In [276]:
pl.sign_in(username='jamesweichert', api_key='PUK0jPsKi3P0P9SVW2x1')

In [277]:
pl.iplot(g, filename='SenateVotingTest')